## Neural Networks for Patches
Goals
* Use Generic DNN for classification
* Use Generic DNN for regression
* Use combined CNN architecture for classification
* Use combined CNN architecture for regression

In [1]:
#global imports
import numpy as np
import tensorflow as tf
import tensorflow.contrib.learn as skflow

In [3]:
from util import PatchesDataLoader

### Data Imports

In [4]:
data_path = '/home/jacky/ws/patches/data/'
pdl = PatchesDataLoader(0.25, data_path, [i for i in range(1)])

In [6]:
def one_hot(types, labels):
    result = np.zeros((labels.shape[0], len(types)))
    for i in range(labels.shape[0]):
        result[i][types[labels[i]]] = 1
    return result        

In [7]:
fc_tr_1h = one_hot([0, 1], pdl.labels['fc']['tr'])
fc_t_1h = one_hot([0, 1], pdl.labels['fc']['t'])

In [9]:
pfc_10_tr = pdl.labels['pfc_10']['tr'].reshape(-1, 1)
pfc_10_t = pdl.labels['pfc_10']['t'].reshape(-1, 1)

### DNN For Classification

In [9]:
# Create model
def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [10]:
def get_batch(data, i, batch_size):
    n = data.shape[0]
    start = i*batch_size
    end = min(n, (i+1)*batch_size)
    return data[start:end]

In [20]:
def print_accuracy(accuracy, tr_data, tr_labels, t_data, t_labels, title, x, y):
    tr_accu = accuracy.eval({x: tr_data, y: tr_labels})
    t_accu = accuracy.eval({x: t_data, y: t_labels})
    print "{0} training accuracy {1}, test accuracy {2}".format(title, tr_accu, t_accu)

def print_accuracy_sk(predictor, tr_data, tr_labels, t_data, t_labels, title):
    tr_accu = predictor.score(tr_data, tr_labels)
    t_accu = predictor.score(t_data, t_labels)
    
    print "{0} train accuracy {1}, test accuracy {2}".format(title, tr_accu, t_accu)

In [12]:
def train(optimizer, cost, accuracy, tr_data, tr_labels, t_data, t_labels, x, y):
    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(tr_data.shape[0]/batch_size)
            for i in range(total_batch):
                batch_x, batch_y = get_batch(tr_data, i, batch_size), get_batch(tr_labels, i, batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
                avg_cost += c / total_batch
            if epoch % display_step == 0:
                print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)
        print "Optimization Finished!"

        print_accuracy(accuracy, tr_data, tr_labels, t_data, t_labels, "DNN", x, y)        

In [13]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100
display_step = 2

# Network Parameters
n_hidden_1 = 500 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_hidden_3 = 256 # 2nd layer number of features
n_input = 474
n_classes = 2

# tf Graph input
x = tf.placeholder("float", [None, n_input], name='x')
y = tf.placeholder("float", [None, n_classes], name='y')

In [14]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = multilayer_perceptron(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.initialize_all_variables()

In [15]:
loss_classifier = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy_clf = tf.reduce_mean(tf.cast(loss_classifier, "float"))

In [21]:
train(optimizer, cost, accuracy_clf, X_tr, fc_tr_1h, X_t, fc_t_1h, x, y)

Epoch: 0001 cost= 14.644018516
Epoch: 0003 cost= 4.101628114
Epoch: 0005 cost= 3.021985632
Epoch: 0007 cost= 2.194346967
Epoch: 0009 cost= 1.881359585
Epoch: 0011 cost= 1.527652933
Epoch: 0013 cost= 1.514903111
Epoch: 0015 cost= 1.228939917
Epoch: 0017 cost= 1.283466823
Epoch: 0019 cost= 1.055163423
Optimization Finished!
DNN training accuracy 0.909541845322, test accuracy 0.890934109688


In [65]:
reg = skflow.TensorFlowDNNRegressor(hidden_units=[400, 300, 200], batch_size=batch_size, steps=10000,
                                    optimizer="Adam", learning_rate=0.001, dropout=0.25, continue_training=True)

In [66]:
reg.fit(X_tr, pfc_10_tr)

Step #99, avg. train loss: 0.06888
Step #199, avg. train loss: 0.03039
Step #299, avg. train loss: 0.02779
Step #400, epoch #1, avg. train loss: 0.02666
Step #500, epoch #1, avg. train loss: 0.02580
Step #600, epoch #1, avg. train loss: 0.02471
Step #700, epoch #2, avg. train loss: 0.02399
Step #800, epoch #2, avg. train loss: 0.02364
Step #900, epoch #2, avg. train loss: 0.02231
Step #1000, epoch #2, avg. train loss: 0.02170
Step #1100, epoch #3, avg. train loss: 0.02122
Step #1200, epoch #3, avg. train loss: 0.02084
Step #1300, epoch #3, avg. train loss: 0.02039
Step #1400, epoch #4, avg. train loss: 0.02045
Step #1500, epoch #4, avg. train loss: 0.02042
Step #1600, epoch #4, avg. train loss: 0.01971
Step #1700, epoch #4, avg. train loss: 0.01964
Step #1800, epoch #5, avg. train loss: 0.01949
Step #1900, epoch #5, avg. train loss: 0.01927
Step #2000, epoch #5, avg. train loss: 0.01928
Step #2100, epoch #6, avg. train loss: 0.01897
Step #2200, epoch #6, avg. train loss: 0.01890
Step #

TensorFlowDNNRegressor(batch_size=100, clip_gradients=5.0, config=None,
            continue_training=True, dropout=0.25,
            hidden_units=[400, 300, 200], learning_rate=0.001, n_classes=0,
            optimizer='Adam', steps=10000, verbose=1)

In [67]:
print_accuracy_sk(reg, X_tr, pfc_10_tr, X_t, pfc_10_t, "DNN")

DNN train accuracy 0.796626172124, test accuracy 0.759615214799


In [ ]:
print_accuracy_sk(reg, X_tr, fer, X_t, pfc_10_t, "DNN")